In [ ]:
from __future__ import unicode_literals
from xml.etree import ElementTree as Xet
import csv
import pandas as pd
import urllib3
import requests
#pip install requests beautifulsoup4 
import bs4
from bs4 import BeautifulSoup as bs

#https://github.com/LaurenceRawlings/savify
#pip install -U savify
from savify import Savify
from savify.types import Type, Format, Quality
from savify.utils import PathHolder
from savify.logger import Logger

#https://github.com/alexmercerind/youtube-search-python
#pip install youtube-search-python
import youtubesearchpython as yts

#https://github.com/ytdl-org/youtube-dl#embedding-youtube-dl
#https://stackoverflow.com/questions/32482230/how-to-set-up-default-download-location-in-youtube-dl
import youtube_dl
import os

#unused
#pip install music-tag
#%pip install pytube
#from pytube import YouTube

In [ ]:
#Module 1: Playing with html pages
#converts the xml page into a dataframe, filters based on watching/completed and also adds mal link in last column
def MakeCSV(exportedxml):
    xml = Xet.parse(exportedxml)
    csvfile = open('mal_to_csv.csv','w',encoding='utf-8')
    csvfile_writer = csv.writer(csvfile)
    csvfile_writer.writerow(['AnimeTitle','ID','Status'])

    for anime in xml.findall('anime'):
        if(anime):
            animename = anime.find('series_title')
            animeid = anime.find('series_animedb_id')
            animestatus = anime.find('my_status')
            csv_line = [animename.text, animeid.text, animestatus.text]
            csvfile_writer.writerow(csv_line)
    csvfile.close()  

    df = pd.read_csv('mal_to_csv.csv')
    df2= df.dropna()
    filters = ['Completed', 'Watching']
    df3 = df2[df2['Status'].isin(['Completed', 'Watching'])].reset_index()
    del df3['index']
    def make_url(row):
        return 'https://myanimelist.net/anime/' + str(row[1])
    df3['URL'] = df3.apply(make_url, axis = 1)
    return df3

#this function gives the title, season and year of the anime as a list
def AnimeAndSeason(entry_page):
    #title  = entry_page.title.text.replace('\n', ' ').strip(' MyAnimeList.net')
    #title  = title.strip(' -')
    title  = entry_page.title.text.replace('\n', ' ').split(' - ')[0].strip()
    links  = entry_page.select('span > a')
    years  = range(1900,2050)
    season = ''
    year   = 1900
    for row in links:
        test = row.contents[0].split()
        if len(test) == 2 and test[1].isdigit() and int(test[1]) in years:
            season_info = test
            season = season_info[0] + ' ' + season_info[1]
            year = int(season_info[1])
    return(title, season, year)

#function below gives a lil bit fucked list of the category/song/artist for individual mal page
def SongAndArtist(entry_page):
    testing = entry_page.find_all('div', attrs={'class':'di-tc va-t'})
    testing2 = entry_page.find_all('div', attrs={'class':'di-tc va-t borderDark pb4'})
    testing3 = testing + testing2
    SAAlist = []
    for entry in testing3:
        category = entry.h2
        if type(category) == bs4.element.Tag: 
            category_name = category.contents
            SAAlist.append(category_name)

        song_td_set = entry.find_all('td', {'width':'84%'})
        for song_td in song_td_set:
            song_data = song_td.text.replace('\n', ' ').strip()
            row = song_data.split('\xa0') #split song# and song-artist
            for datas in row:
                if ' by ' in datas:
                    row2 = datas.split(' by ') #split song and artist
                    data_entry = [row2[0].strip('\"'), row2[1]]  #join song, artist
                    song_links = song_td.find_all('input')
                    for link in song_links:
                        somelink = link['value']
                        if 'spotify' in somelink:
                            data_entry.append(somelink)
                    SAAlist.append(data_entry)
    return SAAlist

#this combines the previous 2 functions and gives a nice dataframe of each row having all the identifying factors of a song
def CreateSongDF(entry_page):
    aas = AnimeAndSeason(entry_page)
    saa = SongAndArtist(entry_page)
    if aas[0] == 'Gyo (GYO: Tokyo Fish Attack!)': saa[2][0] = 'End Theme'
        
    for line in saa:
        if 'Opening Theme' in line: op_index = saa.index(line)
        if 'Ending Theme' in line: ed_index = saa.index(line)
            
    OP = []
    category = saa[op_index][0]
    for op in range(op_index+1,ed_index):
        temp = saa[op]
        temp.insert(0,category)
        OP.append(temp)

    ED = []
    category = saa[ed_index][0]
    for ed in range(ed_index+1,len(saa)):
        temp = saa[ed]
        temp.insert(0,category)
        ED.append(temp)
    
    def Clean_Blanks(List, Length):
        for item in List:
            if len(item) != Length:
                item.extend(' ')
       
    Clean_Blanks(ED, 4)
    Clean_Blanks(OP, 4)
    song_columns = ['category', 'songname', 'artist', 'link']
    opdb = pd.DataFrame(OP, columns = song_columns)
    eddb = pd.DataFrame(ED, columns = song_columns)
    SongDB = pd.concat([opdb,eddb], axis = 0, ignore_index = True)
    SongDB['extra'] = ''
    return SongDB

#Module 2: Playing with internet for spotify and youtube links
#this creates a new dataframe (withc columns as song metadata and a 'links' column) from the csv containing mal urls
def GetSpotify(malcsv):
    song_table = pd.read_csv(malcsv).reset_index()
    song_table.drop(['Unnamed: 0', 'index'], inplace=True, axis=1)    
    link_table = pd.DataFrame(columns = ['anime', 'season', 'year','category', 'songname', 'artist', 'link', 'extra'])
    for i in song_table.index:
        anime_link = song_table.loc[i,'URL']
        html       = requests.get(anime_link).text #type: ignore       
        mal_page   = bs(html, 'lxml')  
        aas        = AnimeAndSeason(mal_page)

        song_subtable    = CreateSongDF(mal_page)
        anime_subtable   = pd.DataFrame(columns=['anime', 'season', 'year'])
        print('Fetching spotify links for', aas[0])
        
        for i in range(0, len(song_subtable)):
            anime_subtable.loc[i,'anime'], anime_subtable.loc[i,'season'],  anime_subtable.loc[i,'year']   = aas[0], aas[1], aas[2]
        anime_subtable = pd.concat([anime_subtable, song_subtable], axis = 1)
        link_table     = pd.concat([link_table, anime_subtable], axis = 0)
    link_table = link_table.reset_index()
    del link_table['index']
    return link_table

#update the spotify sheet with youtube links as well to get a final sheet with all links
def GetYoutube(spotcsv):
    links.to_csv(spotcsv)
    del link_table['Unnamed: 0']
    for j in link_table.index:
        if 'spotify' in str(link_table.loc[j,'link']) or 'youtube' in str(link_table.loc[j,'link']): continue 

        print(j,'Fetching youtube links for', link_table.loc[j,'anime'],link_table.loc[j,'songname'])
        search_text = link_table.loc[j,'songname'] + ' ' + link_table.loc[j,'artist']
        
        if link_table.loc[j,'extra'] == 'fuckup': search_text = link_table.loc[j,'anime'] + ' ' + link_table.loc[j,'songname']
        if link_table.loc[j,'extra'] == 'fuckup2': search_text = link_table.loc[j,'songname']

        first_result = yts.VideosSearch(search_text, limit = 1).result()
        if first_result['result'] == []: 
            if link_table.loc[j,'extra'] == 'fuckup': link_table.loc[j,'extra'] = 'fuckup2'
            link_table.loc[j,'extra'] = 'fuckup'
            print('youtube search fuckup #')
            continue
            link_table.loc[j,'link'] =  first_result['result'][0]['link']
            link_table.loc[j,'extra'] =  first_result['result'][0]['title']
    link_table['status'] = ''
    return link_table

#function to make all entries in the databases valid filenames
def ValidFile(txt):
    filename = ''
    if '...' in txt:
        txt = txt.strip('...')
    for i in txt:
        if i not in '\/*?<>|"':
            filename = filename + i
        elif i in ':':
            filename = filename + '-'
        else:
            filename = filename + '_'
    return filename

#function to check differences in old and new links, replace links if new link found
def New_Entries(new, old):
    temp = pd.DataFrame(columns = old.columns.tolist())
    for i in new.index:
        link, song, artist  = new.loc[i, 'link'], new.loc[i, 'songname'], new.loc[i, 'artist']   
        for j in old.index:
            if old.loc[j, 'link'] ==  link:
                link = "BUSTED"
                break
            if 'spotify' not in link and old.loc[j, 'songname'] == song and old.loc[j, 'artist'] == artist: 
                new.loc[i,'extra'] = 'yt upgrade to spot'
    
        if link != "BUSTED": 
            print("found a new link:", link)
            temp = temp.append(new.loc[i], ignore_index = True)
    return temp

#Module 3: Playing with external libraries to download songs
#downloads spotify links into nested folder properly
def DownloadSpotify(sheet_with_spotify, dl_loc, start, end):
    songset = sheet_with_spotify.iloc[start:end]
    for i in songset.index:
        if 'spotify' not in songset.loc[i,'link'] or songset.loc[i,'status'] == 'Downloaded': continue
        #https://developer.spotify.com/dashboard/applications/f2d89a80d48e41e19e4263d5b3792c21 for api credentials
        sp_link = songset.loc[i,'link']
        year    = str(songset.loc[i,'year'])
        season  = str(songset.loc[i,'season'])
        oped    = songset.loc[i,'category']
        anime   = ValidFile(songset.loc[i,'anime'])
        title   = ValidFile(songset.loc[i,'songname'].strip())
        artist  = ValidFile(songset.loc[i,'artist'].strip())
        path    = year + '/' + season + '/' + anime + '/' + oped + '/' + artist + '/'+ title
        ffpath  = 'ffmpeg'
        dlpath  = dl_loc +'/' + 'Animusic'
        api1    = 'f2d89a80d48e41e19e4263d5b3792c21'
        api2    = '12f6688e93504599b3d0403e8a4f2cc8'
        logger  = Logger(log_location = dlpath, log_level = None)
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print(i, path)
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        
        #C:\Users\dell\AppData\Roaming\Savify\temp is where the temporary download goes
        s = Savify(api_credentials  = (api1,api2), 
                    download_format = Format.MP3, 
                    ffmpeg_location = ffpath, 
                    skip_cover_art  = False, 
                    path_holder = PathHolder(downloads_path=dlpath), 
                    group   = path, 
                    quality = Quality.BEST,
                    logger  = logger)
        s.download(sp_link)
        sheet_with_spotify.loc[i,'status'] = 'Downloaded'

#downloads youtube video but sus 
def DownloadYoutube(sheet_with_youtube, dl_loc, start, end):
    songset2 = sheet_with_youtube.loc[start:end,:]
    for i in songset2.index:
        if 'youtube' not in songset2.loc[i,'link'] or str(songset2.loc[i,'status']) == 'Downloaded' or str(songset2.loc[i,'status']) == 'sus': continue
        year    = str(songset2.loc[i,'year'])
        season  = str(songset2.loc[i,'season'])
        anime   = songset2.loc[i,'anime']
        oped    = songset2.loc[i,'category']
        title   = songset2.loc[i,'songname']
        artist  = songset2.loc[i,'artist']
        yt_link = songset2.loc[i,'link']
        path    = dl_loc + '/' + year + '/' + season + '/' + anime + '/' + oped + '/' + artist + '/'+ title + '/'
        def my_hook(d):
            if d['status'] == 'finished':
                print('~~~~~~', i, title, artist,'~~~~~~')

        ydl_opts = {
            'format': 'bestaudio',
            'outtmpl': path + '%(title)s.%(ext)s',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',}],
                'progress_hooks': [my_hook]}

        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([yt_link])
        sheet_with_youtube.loc[i,'status'] = 'Downloaded'
    return sheet_with_youtube

#Module 4: Playing with datasets to ensure youtube links are right
#returns the 'dataset' set with indexes between 'start' and 'end' updated with the new 'status'
def manual_status_update(dataset, start, end, status)
    yt = pd.read_csv(dataset)
    del yt['Unnamed: 0']
    for i in range(start, end):
        if yt.loc[i,'status'] == 'sus': continue
        yt.loc[i,'status'] = status
    return yt

In [ ]:
##converting xml to a csv with eligible anime and their mal links

MakeCSV('mal_export.xml').to_csv('mal_to_csv_filtered.csv')

In [ ]:
##creating a new csv of all songs and spotify links

GetSpotify('mal_to_csv_filtered.csv').to_csv('sheet_w_spot.csv')

In [ ]:
##filling in gaps with youtube links

yt_csv = GetYoutube('sheet_w_spot.csv')
#yt_csv = GetYoutube('yt_whole_spot_whole.csv') #uncomment for reruns
yt_csv.to_csv('sheet_w_spot_w_yt.csv')

In [ ]:
##downloading songs from spotify

dl_spot = pd.read_csv('sheet_w_spot_w_yt.csv')
#dl_spot = pd.read_csv('sheet_d_spot_w_yt.csv') #uncomment for reruns
del dl_spot['Unnamed: 0']

DownloadSpotify(dl_spot, 'F:/', 0, 0)
dl_spot.to_csv('sheet_d_spot_w_yt.csv')

In [ ]:
##classifying problematic songs as sus
yt = pd.read_csv('sheet_d_spot_w_yt.csv')
del sheet['Unnamed: 0']

yt = pd.DataFrame(columns = sheet.columns.tolist())
for i in sheet.index:
    if 'spotify' in sheet['link'][i]: continue
    if  yt.loc[i, 'extra'] == 'fuckup': yt.loc[i, 'status'] = 'sus'

    red_set = ['cover', 'Cover', 'Nightcore', 'nightcore', 'Instrumental', 'instrumental']
    if any(red in yt.loc[i, 'extra'] for red in red_set): yt.loc[i, 'status'] = 'sus'
   
    red_index = [52,range(62,83),range(115,119),121,130,131,146,147,148,176,200,range(219,225),236,range(242,243),range(266,268),280,282,304,316,352,359,362,376,range(390,401),437,range(458,460),494,521,522,529,range(562,567),570,573,601,613,625,632,range(644,650),range(652,654),664,665]
    #if i in red_index: yt.loc[i, 'status'] = 'sus'

yt.to_csv('sheet_d_spot_s_yt.csv')

In [ ]:
##downloading all songs from youtube links
yt = pd.read_csv('sheet_d_spot_s_yt.csv')
#dl_spot = pd.read_csv('sheet_d_spot_d_yt.csv') #uncomment for reruns
del yt['Unnamed: 0']
DownloadYoutube(yt, 'F:/Animusic', 0, 50)
yt.to_csv('sheet_d_spot_d_yt.csv')

#manual_status_update('sheet_d_spot_d_yt.csv', 0, 13, 'Downloaded').to_csv('sheet_d_spot_d_yt.csv')


In [ ]:
##Ensuring robustness for new database, i.e. everytime a new mal list is exported
##How to introduce insert songs from sources other than mal?

In [ ]:
yt = pd.read_csv('sheet_d_spot_w_yt_copy.csv')
del yt['Unnamed: 0']
i = 40
yt.loc[i, 'link']  = 'https://www.youtube.com/watch?v=xYUzwwPIsnQ'
yt.loc[i, 'extra'] = '五等分の軌跡'

i = 
yt.loc[i, 'link'], yt.loc[i, 'extra'] = 

i = 
yt.loc[i, 'link'], yt.loc[i, 'extra'] = 

i = 
yt.loc[i, 'link'], yt.loc[i, 'extra'] = 

i = 
yt.loc[i, 'link'], yt.loc[i, 'extra'] = 

i = 
yt.loc[i, 'link'], yt.loc[i, 'extra'] = 

i = 
yt.loc[i, 'link'], yt.loc[i, 'extra'] = 

i = 
yt.loc[i, 'link'], yt.loc[i, 'extra'] = 

i = 
yt.loc[i, 'link'], yt.loc[i, 'extra'] = 

yt.to_csv('sheet_d_spot_w_yt_copy.csv')
